In [42]:
# for imports
import random 

In [43]:
class Store:
    def __init__(self, store_id, store_name, branch, estimated_bags, rating, longitude, latitude):
        # private attributes
        self._store_id = store_id
        self._store_name = store_name
        self._branch = branch 
        self._rating = rating 
        self._price_per_bag = None 
        self._estimated_bags = estimated_bags # average_bags_at_9AM
        self._actual_bags = None # actual bags at night
        self._bags_reserved = 0 
        self._bags_canceled = 0 
        self._bags_sold = 0 
        self._longitude = longitude
        self._latitude = latitude

        # WHAT WE MIGHT ADD LATER:
        # cancellation rate history
        # location

    # setters
    def set_price_per_bag(self, price):
        self._price_per_bag = price 

    def set_actual_bags(self, count):
        self._actual_bags = count 


    # getters
    def get_store_id(self):
        return self._store_id
    
    def get_store_name(self):
        return self._store_name
    
    def get_branch(self):
        return self._branch
    
    def get_price_per_bag(self):
        return self._price_per_bag 

    def get_estimated_bags(self):
        return self._estimated_bags 

    def get_actual_bags(self):
        return self._actual_bags 
    
    def get_unreserved_bags(self):
        return self._estimated_bags - self._bags_reserved
    
    def get_rating(self):
        return self._rating
    
    def get_location(self):
        return {self._longitude, self._latitude}

    
    def reserve_bag(self):
        self._bags_reserved += 1

    def cancel_bag(self):
        self._bags_canceled += 1

    def sell_bag(self):
        self._bags_sold += 1


In [44]:
class Customer:
    def __init__(self, customer_id, valuations, longitude, latitude):
        self._customer_id = customer_id
        self._valuations = valuations
        self._longitude = longitude
        self._latitude = latitude
        self._reserved_store_ID = None # store the store at which the customer reserved a bag from for future purposes (and printing purposes too)


    def get_customer_id(self):
        return self._customer_id
    
    def get_valuation(self, store_id):
        return self._valuations[store_id]
    
    def get_location(self):
        return {self._longitude, self._latitude}
    
    def get_reserved_store(self):
        return self._reserved_store_ID
    
    def reserve(self, store_id):
        self._reserved_store_ID = store_id


    

In [45]:
class Marketplace:
    def __init__(self, n = 10, stores = None, customers = None): #either initialize with existing lists or add them later
        self._stores = []
        self._customers = [] 

        # default n is 10 for now, we can change the default later
        self._n = n 

        self.algorithm1 = Algorithm1()

    
    def add_store(self, store):
        self._stores.append(store)

    def add_customer(self, customer):
        self._customers.append(customer)

    def get_stores(self):
        return self._stores
    
    def get_customers(self):
        return self._customers
    

    def select(self, customer): 
        """
        This is the main algorithm that generates the n stores to display.

        Inputs: 
            customer -> customer we are choosing for

        Output: a list of n Store objects
        """
        # for the purposes of milestone 1 simulation, return all stores
        return self._stores

        #return self._greedy_baseline()
        # return algorithm1.select(self._stores, self._n, customer)


    def _greedy_baseline(self): 
      # select just the top-n popular stores (based on ranking ONLY)
      return None

    
        

In [46]:
class Algorithm1:
    def __init__(self):
        pass


    def select(self, stores, n, customer):
        selected_stores = []
        # First, check the stores that the customer rated 3+ that still have unreserved bags today
        liked_stores = self._get_high_valuation_stores(self, stores, customer) 
        
        selected_stores += liked_stores

        # IN ALL THE FUNCTIONS, WE ONLY CONSIDER STORES THAT HAVE UNRESERVED BAGS 


        # if number of liked stores is exactly n, then we just return those (selection function already sorts them)
        if len(liked_stores) == n:
             return selected_stores
        
        # if number of liked stores = 0 (for example, none of the stores they liked still has unreserved or its their first time using the app)
        # we use the main selection function (we will call it the general function) to select based on store rating, price, location and thats it
        if len(liked_stores) == 0:
            selected_stores = self._select_general(self, n, stores, customer)
            return selected_stores
            

        # if the number of liked stores < n, we also use the general function to fill the rest of n
        # we get x = (n - number of liked stores)
        # we select x those based on store rating, price, location
        if len(liked_stores) < n:
            x = self._n - len(liked_stores)
            selected_stores += self._select_general(self, x, stores, customer)
            return selected_stores
        

        # final scenario: if number of liked stores exceeds n, we use a function that takes into consideration the customer's valuation
        # we will call this function the valued selection. It is different than the general function in that it additionally uses valuation 
        # (as well as store rating, price, location)
        # for now, we will only consider the subarray of stores they liked
        # FLAW HERE!!! in this case user will not get the chance to try new stores
        if len(liked_stores) > n:
            selected_stores = self._select_valued(self, n, customer, liked_stores)
            return selected_stores


    def _get_high_valuation_stores(self, stores, customer):
            # first we have an empty array
            # for each store in self._stores
                # if store.get_unreserved_bags() > 0
                # if customer.get_valuation(store) >= 3
                    # add store to array

            # sort those stores based on either price, location, or valuation (need to decide on this)
            # return array
            pass


    def _select_general(self, x, stores, customer):
        # Implement an algorithm using these factors:
            # store.get_rating() for each store in self._stores array
            # store.get_price_per_bag() for each store in self._stores array
            # store.get_location() for each store in self._stores array (returns {longitude, latitude})

        # dont forget to first check if a store has store.get_unreserved_bags() > 0 before considering it
        pass 
    

In [47]:
class Algorithm2: # menna's strategy
    pass


In [48]:
class Algorithm3: # nadine's strategy
    pass

In [49]:
class Algorithm4: # ziad's strategy
    pass

In [50]:
def main():
    # MAIN FUNCTION!!!!!!!!!!!!!!
    # here is where we will simulate
    basic_simulation(100, 15) # simulate for 100 customers and 30 stores


def basic_simulation(customer_count = 10, store_count = 5): # basic simulation for milestone 1
    try:
        # disclaimer: sample stores and customers generated by AI

        # generate random stores and customers
        # return all of the stores for each customer
        app = Marketplace()

        # generate stores
        store_id = 1 ## start with 1 and increment by 1 for each store id
        for i in range(store_count):
            name = f"Store {store_id}"
            branch = f"Branch {random.randint(1,8)}"
            
            estimated_bags = random.randint(5, 15)
            rating = round(random.uniform(3.0, 5.0), 2)

            longitude = 30 + random.random() 
            latitude  = 31 + random.random()  

            app.add_store(
                Store(
                    store_id = store_id,
                    store_name = name,
                    branch = branch,
                    estimated_bags = estimated_bags,
                    rating = rating,
                    longitude = longitude,
                    latitude = latitude
                )
            )
            store_id += 1


        # generate customers
        for i in range(customer_count):
            valuations = {id: random.randint(1,5) for id in range(1, store_id)}
            longitude = 30 + random.random()/5
            latitude = 31 + random.random()/5

            app.add_customer(
                Customer(
                    customer_id = 100+i, # start with id 100 and increment by 1 for each customer
                    valuations = valuations,
                    longitude = longitude,
                    latitude = latitude
                )
            )

        print("=== SIMULATION START ===\n")

        # Simulate display
        for customer in app.get_customers():
            # print(f"Customer {customer.get_customer_id()} views the following stores:")

            selected_stores = app.select(customer)

            # for i, store in enumerate(selected_stores, start = 1):
            #     print(f"{i}: {store.get_store_name()}")

            # simulate reservations
            chosen_store = random.choice(selected_stores)

            if chosen_store.get_unreserved_bags() > 0:
                chosen_store.reserve_bag()
                customer.reserve(chosen_store.get_store_id())
                # print(f" --> Customer {customer.get_customer_id()} RESERVED from {chosen_store.get_store_name()}")
            # else:
                # print(f" --> Customer {customer.get_customer_id()} FAILED to reserve (no bags left)")

            # print()

        print("=== END OF DAY SUMMARY ===\n")
        for store in app.get_stores():
            print(f"{store.get_store_name()} | Reserved: {store._bags_reserved}/{store.get_estimated_bags()}")

        print("\nCustomer reservations:")
        for customer in app.get_customers():
            reserved_store = customer.get_reserved_store()
            if reserved_store is None:
                print(f"Customer {customer.get_customer_id()} FAILED to reserve a store")
            else:
                print(f"Customer {customer.get_customer_id()} reserved from store: {reserved_store}")

        print("\n=== SIMULATION END ===")

    except Exception as e:
        print(f"Error Running Simulation: {e}")


if __name__ == "__main__":
    main()

=== SIMULATION START ===

=== END OF DAY SUMMARY ===

Store 1 | Reserved: 3/9
Store 2 | Reserved: 6/6
Store 3 | Reserved: 7/10
Store 4 | Reserved: 10/10
Store 5 | Reserved: 8/13
Store 6 | Reserved: 5/11
Store 7 | Reserved: 7/9
Store 8 | Reserved: 5/5
Store 9 | Reserved: 7/7
Store 10 | Reserved: 3/8
Store 11 | Reserved: 7/7
Store 12 | Reserved: 5/12
Store 13 | Reserved: 5/5
Store 14 | Reserved: 9/13
Store 15 | Reserved: 6/10

Customer reservations:
Customer 100 reserved from store: 11
Customer 101 reserved from store: 6
Customer 102 reserved from store: 1
Customer 103 reserved from store: 7
Customer 104 reserved from store: 4
Customer 105 reserved from store: 5
Customer 106 reserved from store: 2
Customer 107 reserved from store: 5
Customer 108 reserved from store: 6
Customer 109 reserved from store: 4
Customer 110 reserved from store: 2
Customer 111 reserved from store: 7
Customer 112 reserved from store: 10
Customer 113 reserved from store: 4
Customer 114 reserved from store: 15
Custo